In [56]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pickle
import pandas as pd
import re

In [107]:
# loop with this url to pull all users
user_list_url = 'https://www.anime-planet.com/anime/death-note/watched?page=1'

# use this url to loop through and get all anime titles
animes_url = 'https://www.anime-planet.com/anime/all?page=1'

In [11]:
# Function to parse through the url
def getAndParseUrl(url):
    results = requests.get(url)
#     time.sleep(1)
    soup = BeautifulSoup(results.text, 'html.parser')
    return soup

#### Get a majority of users

In [12]:
# Hard-coded function to loop through each page with user
def get_users():
    all_users = []
    for pg in tqdm(range(1,2945)):
        url = f'https://www.anime-planet.com/anime/death-note/watched?page={pg}'
        page = getAndParseUrl(url)
        users = page.select('td a[href]')
        for i in range(len(users)):
            all_users.append(users[i].text)
    return all_users

This saves the users list as a separate file:

In [13]:
# users = get_users()

In [47]:
# with open("users.pickle", "wb") as fp:
#     pickle.dump(users, fp)

Reload users list

In [165]:
with open('users.pickle', 'rb') as infile:
    users=pickle.load(infile)

infile.close()

In [166]:
users

['000000000000L',
 '0000Dragonfly0000',
 '001heeroyuy',
 '004',
 '0063gam',
 '007Hughy',
 '008Seravee',
 '00anwi',
 '00Canti',
 '00CrowChan',
 '00hank00',
 '00Hei',
 '00HiZaM',
 '00hughel',
 '00ooabhijithoo00',
 '00raiser',
 '00Ryuk00',
 '00shked00',
 '00Venenum00',
 '00X00',
 '00Xander00',
 '01000000',
 '01010011',
 '011000000110',
 '0151Master',
 '018',
 '01Lalo',
 '01Z555',
 '024',
 '029misaki',
 '02adityajain',
 '02Baba',
 '035325',
 '03mjung',
 '0423fmm',
 '04wasabi',
 '05072003',
 '0521neeria',
 '05aletur',
 '05Kaboom',
 '0612007505',
 '069JLU',
 '06m05',
 '072ashatan',
 '074320',
 '0746',
 '07722326502',
 '0773h',
 '0788',
 '07mak',
 '07ManojP',
 '0800808080',
 '0800unicornio',
 '0808',
 '08935730249582347503',
 '08kellerk',
 '09876654321',
 '09astro27nm',
 '09Julix',
 '09lawso',
 '0AbsolutZero0',
 '0airlegend0',
 '0AkiraChan0',
 '0akito0',
 '0Amai0',
 '0anc',
 '0Azrael',
 '0BloodyRose',
 '0bsidian',
 '0bsidion',
 '0Chan',
 '0ctober333',
 '0dark',
 '0dyssey',
 '0eddead0',
 '0Flo

#### Get all anime titles

In [155]:
# Hard coded function to get all anime titles and their urls
def get_animes():
    titles = []
    links = []
    for i in tqdm(range(1,388)):
        animes = getAndParseUrl(f'https://www.anime-planet.com/anime/all?page={i}')
        anime_links = animes.findAll('a', attrs={'class' : 'tooltip'})
        for j in range(2,len(anime_links)):
            links.append('anime-planet.com'+anime_links[j]['href'])
        titles.extend([title.text for title in animes.select('h2')])
    return dict(zip(titles,links))

This saves animes list into a separate file (only run once):

In [149]:
# animes = get_animes()

100%|██████████| 387/387 [04:43<00:00,  1.39it/s]


In [152]:
# with open('animes.pickle', 'wb') as outfile:
#     pickle.dump(animes, outfile)

Reload animes list

In [163]:
with open('animes.pickle', 'rb') as infile:
    animes=pickle.load(infile)

infile.close()

In [167]:
animes

{'Fullmetal Alchemist: Brotherhood': 'anime-planet.com/anime/fullmetal-alchemist-brotherhood',
 'your name.': 'anime-planet.com/anime/your-name',
 'Attack on Titan 3rd Season: Part II': 'anime-planet.com/anime/attack-on-titan-3rd-season-part-ii',
 'A Silent Voice': 'anime-planet.com/anime/a-silent-voice',
 'Haikyuu!! Karasuno High School vs Shiratorizawa Academy': 'anime-planet.com/anime/haikyuu-karasuno-high-school-vs-shiratorizawa-academy',
 'Demon Slayer: Kimetsu no Yaiba': 'anime-planet.com/anime/demon-slayer-kimetsu-no-yaiba',
 'Haikyuu!! Second Season': 'anime-planet.com/anime/haikyuu-second-season',
 'My Hero Academia 3': 'anime-planet.com/anime/my-hero-academia-3',
 'Hunter x Hunter (2011)': 'anime-planet.com/anime/hunter-x-hunter-2011',
 'Mob Psycho 100 II': 'anime-planet.com/anime/mob-psycho-100-ii',
 'Gintama Kanketsu-hen: Yorozuya yo Eien Nare': 'anime-planet.com/anime/gintama-kanketsu-hen-yorozuya-yo-eien-nare',
 'Gintama (2015)': 'anime-planet.com/anime/gintama-2015',
 'M

## Get ratings from each user for each anime

In [178]:
anime_urls = ['https://www.' + url + '/watched' for url in animes.values()]

In [180]:
fma = getAndParseUrl(anime_urls[0])

In [212]:
fma_pgs = fma.findAll('ul', attrs={'class' : 'nav'})
int(fma_pgs[0].findAll('a')[-2].text)

1843

In [ ]:
for i in range(1, int(fma_pgs[0].findAll('a')[-2].text) +1):
    